In [39]:
# Import required libraries
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
import snowflake.connector
from openai import AzureOpenAI
from typing import List, Dict
import logging

In [2]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [60]:
## ignore this now
# class SnowflakeFailureTester:
#     def __init__(self):
#         """
#         Initialize connections to Snowflake and Azure OpenAI using environment variables
#         """
#         # Load Snowflake credentials from environment variables
#         self.snow_conn = snowflake.connector.connect(
#             user=os.getenv('SNOWFLAKE_USER'),
#             password=os.getenv('SNOWFLAKE_PASSWORD'),
#             account=os.getenv('SNOWFLAKE_ACCOUNT'),
#             warehouse=os.getenv('SNOWFLAKE_WAREHOUSE'),
#             database=os.getenv('SNOWFLAKE_DATABASE'),
#             schema=os.getenv('SNOWFLAKE_SCHEMA')
#         )
        
#         # Load Azure OpenAI credentials from environment variables
#         self.openai_client = AzureOpenAI(
#             api_key=os.getenv('AZURE_OPENAI_API_KEY'),
#             api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
#             azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT')
#         )
        
#         self.failure_scenarios = {
#             'data_type_mismatch': self._generate_type_mismatch_data,
#             'out_of_range_values': self._generate_out_of_range_data,
#             'null_violations': self._generate_null_violations,
#             'unique_constraint_violations': self._generate_unique_violations,
#             'foreign_key_violations': self._generate_fk_violations
#         }
        
#     def get_table_metadata(self, table_name: str) -> Dict:
#         """
#         Fetch table metadata from Snowflake
#         """
#         cursor = self.snow_conn.cursor()
#         try:
#             # Get column information
#             cursor.execute(f"""
#                 SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE, CHARACTER_MAXIMUM_LENGTH,
#                        NUMERIC_PRECISION, NUMERIC_SCALE
#                 FROM INFORMATION_SCHEMA.COLUMNS
#                 WHERE TABLE_NAME = '{table_name}'
#                 ORDER BY ORDINAL_POSITION
#             """)
#             columns = cursor.fetchall()
            
#             # Get constraints information
#             cursor.execute(f"""
#                 SELECT CONSTRAINT_TYPE, CONSTRAINT_NAME, COLUMN_NAME
#                 FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc
#                 JOIN INFORMATION_SCHEMA.CONSTRAINT_COLUMN_USAGE cc 
#                     ON tc.CONSTRAINT_NAME = cc.CONSTRAINT_NAME
#                 WHERE tc.TABLE_NAME = '{table_name}'
#             """)
#             constraints = cursor.fetchall()
            
#             return {
#                 'columns': columns,
#                 'constraints': constraints
#             }
#         finally:
#             cursor.close()

#     def _generate_type_mismatch_data(self, metadata: Dict) -> pd.DataFrame:
#         """
#         Generate data with intentional type mismatches
#         """
#         prompt = f"""
#         Generate 5 rows of data that would cause type mismatch errors for the following columns:
#         {json.dumps(metadata['columns'])}
#         Format the response as a JSON array of objects with intentional type mismatches.
#         """
        
#         response = self.openai_client.chat.completions.create(
#             model="gpt-4",
#             messages=[{"role": "user", "content": prompt}]
#         )
        
#         data = json.loads(response.choices[0].message.content)
#         return pd.DataFrame(data)

#     def _generate_out_of_range_data(self, metadata: Dict) -> pd.DataFrame:
#         """
#         Generate data with out-of-range values
#         """
#         prompt = f"""
#         Generate 5 rows of data with out-of-range values for numeric and date columns:
#         {json.dumps(metadata['columns'])}
#         Include values that exceed maximum/minimum bounds for numeric types,
#         and invalid dates for date types. Format as JSON array.
#         """
        
#         response = self.openai_client.chat.completions.create(
#             model="gpt-4",
#             messages=[{"role": "user", "content": prompt}]
#         )
        
#         data = json.loads(response.choices[0].message.content)
#         return pd.DataFrame(data)

#     def _generate_null_violations(self, metadata: Dict) -> pd.DataFrame:
#         """
#         Generate data with NULL values for non-nullable columns
#         """
#         prompt = f"""
#         Generate 5 rows of data with NULL values for non-nullable columns:
#         {json.dumps(metadata['columns'])}
#         Format as JSON array with strategic NULL placements.
#         """
        
#         response = self.openai_client.chat.completions.create(
#             model="gpt-4",
#             messages=[{"role": "user", "content": prompt}]
#         )
        
#         data = json.loads(response.choices[0].message.content)
#         return pd.DataFrame(data)

#     def _generate_unique_violations(self, metadata: Dict) -> pd.DataFrame:
#         """
#         Generate data that violates unique constraints
#         """
#         cursor = self.snow_conn.cursor()
#         try:
#             # First get some existing values
#             unique_columns = [c[2] for c in metadata['constraints'] 
#                             if c[0] == 'UNIQUE']
#             if unique_columns:
#                 column_list = ', '.join(unique_columns)
#                 cursor.execute(f"SELECT {column_list} FROM {table_name} LIMIT 5")
#                 existing_values = cursor.fetchall()
                
#                 prompt = f"""
#                 Generate 5 rows of data that violate unique constraints by reusing these values:
#                 Existing values: {existing_values}
#                 Unique columns: {unique_columns}
#                 Format as JSON array.
#                 """
                
#                 response = self.openai_client.chat.completions.create(
#                     model="gpt-4",
#                     messages=[{"role": "user", "content": prompt}]
#                 )
                
#                 data = json.loads(response.choices[0].message.content)
#                 return pd.DataFrame(data)
#         finally:
#             cursor.close()

#     def _generate_fk_violations(self, metadata: Dict) -> pd.DataFrame:
#         """
#         Generate data that violates foreign key constraints
#         """
#         # Similar to unique violations but for foreign keys
#         # Would need to analyze referenced tables as well
#         pass

#     def test_table(self, table_name: str, scenario: str = 'all') -> Dict:
#         """
#         Run failure tests on a specific table
#         """
#         results = {
#             'table_name': table_name,
#             'timestamp': datetime.now().isoformat(),
#             'scenarios_tested': [],
#             'failures': []
#         }
        
#         metadata = self.get_table_metadata(table_name)
        
#         scenarios = [scenario] if scenario != 'all' else self.failure_scenarios.keys()
        
#         for scenario_name in scenarios:
#             try:
#                 if scenario_name in self.failure_scenarios:
#                     logger.info(f"Testing scenario: {scenario_name}")
                    
#                     # Generate test data
#                     test_data = self.failure_scenarios[scenario_name](metadata)
                    
#                     # Try to insert the data
#                     cursor = self.snow_conn.cursor()
#                     try:
#                         for _, row in test_data.iterrows():
#                             insert_sql = f"""
#                                 INSERT INTO {table_name}
#                                 ({', '.join(row.index)})
#                                 VALUES ({', '.join(['%s'] * len(row))})
#                             """
#                             try:
#                                 cursor.execute(insert_sql, tuple(row))
#                                 # If we get here, the failure test failed (data was inserted successfully)
#                                 results['failures'].append({
#                                     'scenario': scenario_name,
#                                     'unexpected_success': True,
#                                     'data': row.to_dict()
#                                 })
#                             except Exception as e:
#                                 # This is actually what we want - log the error
#                                 results['failures'].append({
#                                     'scenario': scenario_name,
#                                     'error_message': str(e),
#                                     'data': row.to_dict()
#                                 })
#                     finally:
#                         cursor.close()
                        
#                     results['scenarios_tested'].append(scenario_name)
                    
#             except Exception as e:
#                 logger.error(f"Error in scenario {scenario_name}: {str(e)}")
#                 results['failures'].append({
#                     'scenario': scenario_name,
#                     'error': str(e),
#                     'stage': 'scenario_execution'
#                 })
        
#         return results

#     def test_schema(self, exclude_tables: List[str] = None) -> Dict:
#         """
#         Test all tables in the schema
#         """
#         cursor = self.snow_conn.cursor()
#         try:
#             cursor.execute("""
#                 SELECT TABLE_NAME 
#                 FROM INFORMATION_SCHEMA.TABLES 
#                 WHERE TABLE_SCHEMA = CURRENT_SCHEMA()
#             """)
#             tables = [row[0] for row in cursor.fetchall()]
            
#             if exclude_tables:
#                 tables = [t for t in tables if t not in exclude_tables]
            
#             results = {}
#             for table in tables:
#                 results[table] = self.test_table(table)
            
#             return results
#         finally:
#             cursor.close()

#     def save_results(self, results: Dict, output_path: str):
#         """
#         Save test results to a file
#         """
#         with open(output_path, 'w') as f:
#             json.dump(results, f, indent=2)

## db and api connection testing 


In [61]:
# Import required libraries
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
import snowflake.connector
# from openai import AzureOpenAI
from llama_index.llms.azure_openai import AzureOpenAI
from typing import List, Dict
import logging
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Snowflake connection
def init_snowflake():
    return snowflake.connector.connect(
        user=os.getenv('SNOWFLAKE_USER'),
        password=os.getenv('SNOWFLAKE_PASSWORD'),
        account=os.getenv('SNOWFLAKE_ACCOUNT'),
        warehouse=os.getenv('SNOWFLAKE_WAREHOUSE'),
        database=os.getenv('SNOWFLAKE_DATABASE'),
        schema=os.getenv('SNOWFLAKE_SCHEMA')
    )

# Initialize Azure OpenAI client
def init_openai():
    return AzureOpenAI(
        engine=os.getenv("AZURE_OPENAI_4o_MODEL_NAME"),
        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
        api_version=os.getenv('AZURE_OPENAI_API_VERSION')
    )

try:
    snow_conn = init_snowflake()
    print("Snowflake connection established")
    
    openai_client = init_openai()
    print("Azure OpenAI connection established")
    
except Exception as e:
    print(f"Error establishing connections: {str(e)}")

INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.12.4, Python Version: 3.11.7, Platform: Windows-10-10.0.22631-SP0
INFO:snowflake.connector.connection:Connecting to GLOBAL Snowflake domain
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


Snowflake connection established
Azure OpenAI connection established


## create the metadata retrieval function

In [62]:

def get_table_metadata(conn, table_name: str) -> Dict:
    """Fetch table metadata from Snowflake"""
    cursor = conn.cursor()
    try:
        # Get column information
        cursor.execute(f"""
            SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE, CHARACTER_MAXIMUM_LENGTH,
                   NUMERIC_PRECISION, NUMERIC_SCALE
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_NAME = '{table_name}'
            ORDER BY ORDINAL_POSITION
        """)
        columns = cursor.fetchall()
    

        # cursor.execute(f"""
        #     SELECT CONSTRAINT_TYPE, CONSTRAINT_NAME
        #     FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS
        #     WHERE TABLE_NAME = '{table_name}'
        # """)
        # constraints = cursor.fetchall()
        
        return {
            'columns': columns
            # 'constraints': constraints
        }
    finally:
        cursor.close()

        

try:
    test_table = "USERS"  # need to make this iterative over all tables in given schema
    metadata = get_table_metadata(snow_conn, test_table)
    print(f"Retrieved metadata for table {test_table}")
    print("Columns:", metadata['columns'])
    # print("Constraints:", metadata['constraints'])
except Exception as e:
    print(f"Error retrieving metadata: {str(e)}")

INFO:snowflake.connector.cursor:Number of results in first chunk: 5


Retrieved metadata for table USERS
Columns: [('USER_ID', 'NUMBER', 'NO', None, 38, 0), ('USERNAME', 'TEXT', 'NO', 16777216, None, None), ('EMAIL', 'TEXT', 'NO', 16777216, None, None), ('PASSWORD', 'TEXT', 'NO', 16777216, None, None), ('ROLE', 'TEXT', 'NO', 16777216, None, None)]


## create the data generation functions

In [63]:
def generate_type_mismatch_data(client, metadata: Dict) -> pd.DataFrame:
    """Generate data with intentional type mismatches"""
    prompt = f"""
    Generate 5 rows of data that would cause type mismatch errors for the following columns:
    {json.dumps(metadata['columns'])}
    Format the response as a JSON array of objects with intentional type mismatches.
    """
    
    response = client.predict(prompt)
    data = json.loads(response)
    return pd.DataFrame(data)

def generate_null_violations(client, metadata: Dict) -> pd.DataFrame:
    """Generate data with NULL values for non-nullable columns"""
    prompt = f"""
    Generate 5 rows of data with NULL values for non-nullable columns:
    {json.dumps(metadata['columns'])}
    Format as JSON array with strategic NULL placements.
    """
    
    response = client.predict(prompt)
    data = json.loads(response)
    return pd.DataFrame(data)

# Test data generation
try:
    test_data_type = generate_type_mismatch_data(openai_client, metadata)
    print("Generated type mismatch data:")
    print(test_data_type)
    
    test_data_null = generate_null_violations(openai_client, metadata)
    print("\nGenerated null violation data:")
# Test data generation
# try:
#     # Use the llm instance that's already configured
#     test_data_type = generate_type_mismatch_data(llm, metadata)
#     print("Generated type mismatch data:")
#     print(test_data_type)
    
#     test_data_null = generate_null_violations(llm, metadata)
#     print("\nGenerated null violation data:")
#     print(test_data_null)
except Exception as e:
    print(f"Error generating test data: {str(e)}")

Error generating test data: 1 validation error for LLMPredictStartEvent
template
  Input should be a valid dictionary or instance of BasePromptTemplate [type=model_type, input_value='\n    Generate 5 rows of... type mismatches.\n    ', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type


In [58]:
import pandas as pd
import json
from llama_index.llms.azure_openai import AzureOpenAI

def generate_type_mismatch_data(client, metadata: Dict) -> pd.DataFrame:
    """Generate data with intentional type mismatches"""
    prompt = f"""
    Generate 5 rows of data that would cause type mismatch errors for the following columns:
    {json.dumps(metadata['columns'])}
    Format the response as a JSON array of objects with intentional type mismatches.
    """
    
    # Send the prompt to the Azure OpenAI client
    response = client.predict(prompt)
    
    # Parse the response into JSON
    data = json.loads(response)
    return pd.DataFrame(data)

def generate_null_violations(client, metadata: Dict) -> pd.DataFrame:
    """Generate data with NULL values for non-nullable columns"""
    prompt = f"""
    Generate 5 rows of data with NULL values for non-nullable columns:
    {json.dumps(metadata['columns'])}
    Format as JSON array with strategic NULL placements.
    """
    
    # Send the prompt to the Azure OpenAI client
    response = client.predict(prompt)
    
    # Parse the response into JSON
    data = json.loads(response)
    return pd.DataFrame(data)

# Test data generation
try:
    
    test_data_type = generate_type_mismatch_data(openai_client, metadata)
    print("Generated type mismatch data:")
    print(test_data_type)
    
    test_data_null = generate_null_violations(openai_client, metadata)
    print("\nGenerated null violation data:")
    print(test_data_null)
except Exception as e:
    print(f"Error generating test data: {str(e)}")


Error generating test data: 1 validation error for LLMPredictStartEvent
template
  Input should be a valid dictionary or instance of BasePromptTemplate [type=model_type, input_value='\n    Generate 5 rows of... type mismatches.\n    ', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type


## AI connection working

In [33]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

from llama_index.llms.azure_openai import AzureOpenAI
# from llama_index.legacy.llms.azure_openai import AzureOpenAI

# Initialize the AzureOpenAI instance with the loaded API key and endpoint
## PS. model is not a required attribute for azure openai 
## if you specify azure_deployment, but it affects llm.model attribute in this instance
llm = AzureOpenAI(
    engine=os.getenv("AZURE_OPENAI_4o_MODEL_NAME"), 
    azure_deployment=os.getenv("AZURE_OPENAI_4o_DEPLOYMENT_NAME"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0
)

# Define a simple prompt
prompt = "What is the capital of France?"

# Execute the chain
## PS. if you use from llama_index.legacy.llms.azure_openai import AzureOpenAI
## please change to response = llm.complete(prompt)
r = llm.completion_to_prompt(prompt=prompt)
# response = llm.complete(prompt)

# Display the response from current model
print(f"Response with {llm.azure_deployment}: {r}")

Response with gpt-4o: What is the capital of France?
